In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests

In [2]:
wikiurl = "https://en.wikipedia.org/wiki/List_of_Survivor_(American_TV_series)_contestants"

table_class = "wikitable sortable jquery-tablesorter"

response = requests.get(wikiurl)

200


In [3]:
soup = BeautifulSoup(response.text, 'html.parser')

df1 = pd.read_html(str(soup.find('table', {'class':"wikitable"})))[0]

df2 = pd.read_html(str(soup.find_all('table', {'class':"wikitable"})[1]))[0]

df3 = pd.read_html(str(soup.find_all('table', {'class':"wikitable"})[2]))[0]

df = pd.concat([df1, df2, df3], axis=0)

In [89]:
df_ = (
    df
    .drop(['Name', 'Season'], axis=1)
    .loc[~df['Finish'].isnull()]
    .assign(
        Finish=lambda df_: df_['Finish'].str.replace('Winner', '1th').replace('Runner-Up', '2th').replace('Runner-up', '2th').replace('Co-runner up', '2th').replace('2nd Runner-up', '3th').str.extract('(\d+)').astype(int)
    )
    
)

state_ = (
    df_
    ['Hometown']
    .str.split(',', expand=True)
    [1]
)

df_ = (
    df_
    .assign(state_=state_.str.replace('New York', 'NY').replace('California', 'CA'))
)

In [106]:
(
    df_
    .groupby(['Profession'])
    .agg(
        count_=('Finish', 'count'),
        avg_finish=('Finish', 'mean')
    )
    .sort_values('avg_finish', ascending=True)
    .query("count_ > 2")
)

,count_,avg_finish
Profession,,
Flight Attendant,3,3.666667
Actress,4,5.250000
Pharmaceutical Saleswoman,4,5.500000
Office Assistant,3,5.666667
Advertising Executive,4,5.750000
Medical Student,3,6.000000
Nanny,3,6.333333
Soccer Coach/Musician,3,6.333333
Waiter,3,7.000000


In [98]:
(
    df_
    .loc[df_['Finish'] == 1]
    .groupby(['state_'])
    .agg(
        winners_=('Finish', 'count')
    )
    .sort_values('winners_', ascending=False)
    .query("winners_ >= 2")
)

,winners_
state_,
CA,8
NJ,4
NY,4
PA,3
UT,2
IA,2
TX,2
ON,2


In [100]:
ca_ = 8 / 39.35
nj_ = 4 / 8.885
ny_ = 4 / 19.51
pa_ = 3 / 12.79
ut_ = 2 / 3.151
ia_ = 2 / 3.15
tx_ = 2 / 28.64

In [101]:
{'ca': ca_, 'nj': nj_, 'ny': ny_, 'pa': pa_, 'ut': ut_, 'ia': ia_, 'tx': tx_}

{'ca': 0.20330368487928843,
 'nj': 0.4501969611705121,
 'ny': 0.20502306509482315,
 'pa': 0.23455824863174357,
 'ut': 0.634719136781974,
 'ia': 0.6349206349206349,
 'tx': 0.06983240223463687}

In [96]:
for x in df_['Profession'].value_counts().index:
    print(x)

Attorney
Student
Bartender
Firefighter
Law Student
Police Officer
Real Estate Agent
Teacher
Waitress
Construction Worker
Nurse
Model
Motivational Speaker
Personal Trainer
Pharmaceutical Saleswoman
Advertising Executive
Troubled Teens Mentor
Social Worker
Marketing Manager
Actress
Writer
Hairdresser
Soccer Coach/Musician
Pharmaceutical Sales Representative
Army Veteran
Pre-Med Student
Ex-NFL Player
Gravedigger
Farmer
Executive Recruiter
Photographer
Nanny
Cosmetology Student
Office Assistant
Cattle Rancher
Homemaker
Marketing Director
Physician
Oil Company Owner
Waiter
Hairstylist
Rancher
Medical Student
College Student
Flight Attendant
Fitness Instructor
Accountant
Sex Therapist
Musician
Art Consultant
Mortician
Marketing Executive
Administrative Assistant
Pageant Coach
Television Writer
Economics Student
Truck Driver
Male Model
Jewelry Designer
Barrel Racer
Swimsuit Model
Medical Saleswoman
Internet Projects Manager
Jeweler
School Principal
Actor
Seminary Student
Small Business Owner
